# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math


## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
price = data['latestPrice']
pe_ratio = data['peRatio']


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
            
        ],index = my_columns
            ),
        ignore_index = True)
        



## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)

final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis = 1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,667.310,3.13,N/A
1,HPE,17.881,3.38,N/A
2,COO,428.150,3.62,N/A
3,HPQ,39.230,3.73,N/A
4,EBAY,68.944,3.82,N/A
5,NRG,40.800,4.10,N/A
6,DHI,97.850,4.36,N/A
7,BEN,35.638,4.90,N/A
8,HOLX,71.050,4.96,N/A
9,MPC,73.530,5.04,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row,'Number of Shares to Buy'] = position_size/final_dataframe.loc[row,'Price']
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,667.310,3.13,299.710779
1,HPE,17.881,3.38,11185.056764
2,COO,428.150,3.62,467.126007
3,HPQ,39.230,3.73,5098.139179
4,EBAY,68.944,3.82,2900.905082
5,NRG,40.800,4.10,4901.960784
6,DHI,97.850,4.36,2043.944813
7,BEN,35.638,4.90,5611.987205
8,HOLX,71.050,4.96,2814.919071
9,MPC,73.530,5.04,2719.978240


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [27]:
symbol = 'AAPL'
batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_url).json()

pe_ratio = data[symbol]['quote']['peRatio']

pb_ratio = data[symbol]['advanced-stats']['priceToBook']

ps_ratio = data[symbol]['advanced-stats']['priceToSales']

enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']

ebitda = data[symbol]['advanced-stats']['EBITDA']

gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit



Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [28]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-earnings Ratio',
    'Price-to-earnings Percentile',
    'Price-to-book Ratio',
    'Price-to-book Percentile',
    'Price-to-sales Ratio',
    'Price-to-sales Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        enterpriseVal = data[symbol]['advanced-stats']['enterpriseValue']
        EBITDA = data[symbol]['advanced-stats']['EBITDA']
        GP = data[symbol]['advanced-stats']['grossProfit']
        try:
            evToEbitda = enterpriseVal/EBITDA
        except TypeError:
            evToEbitda = np.NaN
        try:
            evToGp = enterpriseVal/GP
        except TypeError:
            evToGp = np.NaN
        rv_dataframe = rv_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            evToEbitda,
            'N/A',
            evToGp,
            'N/A',
            'N/A'
        ],
        index = rv_columns), 
        ignore_index = True)

In [29]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-earnings Ratio,Price-to-earnings Percentile,Price-to-book Ratio,Price-to-book Percentile,Price-to-sales Ratio,Price-to-sales Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,147.23,N/A,19.11,N/A,4.31,N/A,3.6600,N/A,13.882190,N/A,7.000521,N/A,N/A
1,AAL,19.54,N/A,-4.04,N/A,-1.67,N/A,0.5043,N/A,-7.198120,N/A,1.462381,N/A,N/A
2,AAP,240.28,N/A,24.38,N/A,4.82,N/A,1.3800,N/A,13.547478,N/A,3.099675,N/A,N/A
3,AAPL,181.05,N/A,16.16,N/A,23.14,N/A,4.0000,N/A,12.174008,N/A,9.527918,N/A,N/A
4,ABBV,135.71,N/A,32.36,N/A,18.33,N/A,4.5100,N/A,11.628959,N/A,8.127812,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,141.70,N/A,27.46,N/A,-5.13,N/A,6.4900,N/A,22.911687,N/A,10.609644,N/A,N/A
501,ZBH,134.11,N/A,33.29,N/A,2.13,N/A,3.4400,N/A,18.875993,N/A,6.019007,N/A,N/A
502,ZBRA,560.10,N/A,35.40,N/A,10.81,N/A,5.6200,N/A,26.725125,N/A,11.488737,N/A,N/A
503,ZION,72.90,N/A,6.77,N/A,1.53,N/A,3.1800,N/A,5.128123,N/A,2.837441,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [30]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-earnings Ratio,Price-to-earnings Percentile,Price-to-book Ratio,Price-to-book Percentile,Price-to-sales Ratio,Price-to-sales Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,184.700,N/A,62.40,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,290.980,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,327.910,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.04,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,30.620,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.210,N/A,14.74,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,57.800,N/A,32.09,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,36.584,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
193,FRT,135.860,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2765.023,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [31]:
for column in ['Price','Price-to-earnings Ratio','Price-to-book Ratio',
               'Price-to-sales Ratio','EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(),inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [32]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-earnings Ratio,Price-to-earnings Percentile,Price-to-book Ratio,Price-to-book Percentile,Price-to-sales Ratio,Price-to-sales Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [33]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-earnings Ratio':'Price-to-earnings Percentile',
    'Price-to-book Ratio':'Price-to-book Percentile',
    'Price-to-sales Ratio':'Price-to-sales Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile',
    }

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100.0
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-earnings Ratio,Price-to-earnings Percentile,Price-to-book Ratio,Price-to-book Percentile,Price-to-sales Ratio,Price-to-sales Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,147.23,N/A,19.11,0.368317,4.31,0.560396,3.6600,0.532673,13.882190,0.457426,7.000521,0.483168,N/A
1,AAL,19.54,N/A,-4.04,0.051485,-1.67,0.055446,0.5043,0.049505,-7.198120,0.023762,1.462381,0.061386,N/A
2,AAP,240.28,N/A,24.38,0.495050,4.82,0.601980,1.3800,0.189109,13.547478,0.441584,3.099675,0.154455,N/A
3,AAPL,181.05,N/A,16.16,0.299010,23.14,0.942574,4.0000,0.585149,12.174008,0.382178,9.527918,0.671287,N/A
4,ABBV,135.71,N/A,32.36,0.647525,18.33,0.920792,4.5100,0.634653,11.628959,0.354455,8.127812,0.572277,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,141.70,N/A,27.46,0.564356,-5.13,0.051485,6.4900,0.786139,22.911687,0.730693,10.609644,0.722772,N/A
501,ZBH,134.11,N/A,33.29,0.661386,2.13,0.299010,3.4400,0.513861,18.875993,0.623762,6.019007,0.388119,N/A
502,ZBRA,560.10,N/A,35.40,0.706931,10.81,0.833663,5.6200,0.735644,26.725125,0.801980,11.488737,0.762376,N/A
503,ZION,72.90,N/A,6.77,0.095050,1.53,0.170297,3.1800,0.479208,5.128123,0.071287,2.837441,0.138614,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [34]:
from statistics import mean

for row in rv_dataframe.index:
    valuePercentiles = []
    for metric in metrics.keys():
        valuePercentiles.append(rv_dataframe.loc[row,metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(valuePercentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-earnings Ratio,Price-to-earnings Percentile,Price-to-book Ratio,Price-to-book Percentile,Price-to-sales Ratio,Price-to-sales Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,147.23,N/A,19.11,0.368317,4.31,0.560396,3.6600,0.532673,13.882190,0.457426,7.000521,0.483168,0.480396
1,AAL,19.54,N/A,-4.04,0.051485,-1.67,0.055446,0.5043,0.049505,-7.198120,0.023762,1.462381,0.061386,0.048317
2,AAP,240.28,N/A,24.38,0.495050,4.82,0.601980,1.3800,0.189109,13.547478,0.441584,3.099675,0.154455,0.376436
3,AAPL,181.05,N/A,16.16,0.299010,23.14,0.942574,4.0000,0.585149,12.174008,0.382178,9.527918,0.671287,0.576040
4,ABBV,135.71,N/A,32.36,0.647525,18.33,0.920792,4.5100,0.634653,11.628959,0.354455,8.127812,0.572277,0.625941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,141.70,N/A,27.46,0.564356,-5.13,0.051485,6.4900,0.786139,22.911687,0.730693,10.609644,0.722772,0.571089
501,ZBH,134.11,N/A,33.29,0.661386,2.13,0.299010,3.4400,0.513861,18.875993,0.623762,6.019007,0.388119,0.497228
502,ZBRA,560.10,N/A,35.40,0.706931,10.81,0.833663,5.6200,0.735644,26.725125,0.801980,11.488737,0.762376,0.768119
503,ZION,72.90,N/A,6.77,0.095050,1.53,0.170297,3.1800,0.479208,5.128123,0.071287,2.837441,0.138614,0.190891


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [35]:
rv_dataframe.sort_values('RV Score',ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-earnings Ratio,Price-to-earnings Percentile,Price-to-book Ratio,Price-to-book Percentile,Price-to-sales Ratio,Price-to-sales Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.575,N/A,7.30,0.106931,0.5105,0.057426,0.4310,0.033663,3.383972,0.033663,0.405249,0.007921,0.047921
1,AAL,19.540,N/A,-4.04,0.051485,-1.6700,0.055446,0.5043,0.049505,-7.198120,0.023762,1.462381,0.061386,0.048317
2,HPQ,38.700,N/A,3.67,0.069307,-13.2900,0.037624,0.3433,0.013861,4.181607,0.051485,1.793597,0.083168,0.051089
3,PRU,116.804,N/A,6.41,0.089109,0.7177,0.065347,0.6134,0.057426,4.568227,0.057426,0.603686,0.011881,0.056238
4,BEN,36.000,N/A,4.90,0.077228,0.7696,0.071287,1.0300,0.121782,3.236983,0.031683,0.890883,0.023762,0.065149
5,KSS,48.410,N/A,7.77,0.112871,1.4400,0.148515,0.3586,0.021782,3.683075,0.041584,1.162816,0.045545,0.074059
6,BA,220.640,N/A,-14.99,0.041584,-8.7000,0.045545,2.0100,0.311881,-33.032848,0.005941,-1329.284946,0.001980,0.081386
7,DHI,100.140,N/A,4.39,0.075248,1.2300,0.118812,0.6632,0.061386,3.476373,0.035644,2.455616,0.118812,0.081980
8,HPE,17.765,N/A,3.37,0.065347,0.5638,0.061386,0.3980,0.031683,7.681371,0.164356,2.062819,0.104950,0.085545
9,AIG,63.370,N/A,9.78,0.148515,0.7729,0.073267,1.0600,0.126733,4.660231,0.061386,0.994887,0.029703,0.087921


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [36]:
portfolio_input()

Enter the value of your portfolio:10000000


In [37]:
positionSize = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = positionSize/rv_dataframe.loc[row,'Price']
rv_dataframe

/Users/jacobhatef/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-earnings Ratio,Price-to-earnings Percentile,Price-to-book Ratio,Price-to-book Percentile,Price-to-sales Ratio,Price-to-sales Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.575,6999.125109,7.30,0.106931,0.5105,0.057426,0.4310,0.033663,3.383972,0.033663,0.405249,0.007921,0.047921
1,AAL,19.540,10235.414534,-4.04,0.051485,-1.6700,0.055446,0.5043,0.049505,-7.198120,0.023762,1.462381,0.061386,0.048317
2,HPQ,38.700,5167.958656,3.67,0.069307,-13.2900,0.037624,0.3433,0.013861,4.181607,0.051485,1.793597,0.083168,0.051089
3,PRU,116.804,1712.270128,6.41,0.089109,0.7177,0.065347,0.6134,0.057426,4.568227,0.057426,0.603686,0.011881,0.056238
4,BEN,36.000,5555.555556,4.90,0.077228,0.7696,0.071287,1.0300,0.121782,3.236983,0.031683,0.890883,0.023762,0.065149
5,KSS,48.410,4131.377815,7.77,0.112871,1.4400,0.148515,0.3586,0.021782,3.683075,0.041584,1.162816,0.045545,0.074059
6,BA,220.640,906.453952,-14.99,0.041584,-8.7000,0.045545,2.0100,0.311881,-33.032848,0.005941,-1329.284946,0.001980,0.081386
7,DHI,100.140,1997.203915,4.39,0.075248,1.2300,0.118812,0.6632,0.061386,3.476373,0.035644,2.455616,0.118812,0.081980
8,HPE,17.765,11258.091753,3.37,0.065347,0.5638,0.061386,0.3980,0.031683,7.681371,0.164356,2.062819,0.104950,0.085545
9,AIG,63.370,3156.067540,9.78,0.148515,0.7729,0.073267,1.0600,0.126733,4.660231,0.061386,0.994887,0.029703,0.087921


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [38]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')

rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [39]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [40]:
column_formats = {
    'A':['Ticker',string_template],
    'B':['Price',dollar_template],
    'C':['Number of Shares to Buy',integer_template],
    'D':['Price-to-earnings Ratio',float_template],
    'E':['Price-to-earnings Percentile',percent_template],
    'F':['Price-to-book Ratio',float_template],
    'G':['Price-to-book Percentile',percent_template],
    'H':['Price-to-sales Ratio',float_template],
    'I':['Price-to-sales Percentile',percent_template],
    'J':['EV/EBITDA',float_template],
    'K':['EV/EBITDA Percentile',percent_template],
    'L':['EV/GP',float_template],
    'M':['EV/GP Percentile',percent_template],
    'N':['RV Score',percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column( f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [41]:
writer.save()